In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support
import re

In [2]:
df_questions=pd.read_csv('../Maj2NLP/data/Questions.csv', encoding='ISO-8859-1')
df_answers=pd.read_csv('../Maj2NLP/data/Answers.csv', encoding='ISO-8859-1')


In [ ]:
df=df_questions.merge(df_answers,left_on="Id",right_on="ParentId")

In [ ]:
df=df[["Title","Body_y"]]
X_train, X_test, y_train, y_test = train_test_split(df["Title"],df["Body_y"], test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

# Charger les données de questions et de réponses
questions = X_train
answers = y_train

# Prétraitement des données
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

# Convertir les questions et les réponses en séquences numériques
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Appliquer le padding pour que toutes les séquences aient la même longueur
question_sequences = pad_sequences(question_sequences)
answer_sequences = pad_sequences(answer_sequences)

# Entraîner le modèle RNN
embedding_size = 128
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_size, input_length=question_sequences.shape[1]))
model.add(LSTM(64))
model.add(Dense(answer_sequences.shape[1], activation='relu'))
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.fit(question_sequences, answer_sequences, batch_size=128, epochs=100, validation_split=0.2)

Epoch 1/100
 113/4936 [..............................] - ETA: 30:38 - loss: 7478236.5000 - accuracy: 0.0259

ResourceExhaustedError: Graph execution error:

Detected at node 'RMSprop/RMSprop/update/ReadVariableOp' defined at (most recent call last):
    File "c:\Users\bouba\anaconda3\envs\chatbot\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\bouba\anaconda3\envs\chatbot\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\bouba\anaconda3\envs\chatbot\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\bouba\anaconda3\envs\chatbot\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\bouba\anaconda3\envs\chatbot\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\bouba\anaconda3\envs\chatbot\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\bouba\anaconda3\envs\

In [ ]:
# Prétraitement de la question
question = input("Votre question ?")
question_sequence = tokenizer.texts_to_sequences([question])
question_sequence = pad_sequences(question_sequence, maxlen=question_sequences.shape[1])

# Prédiction de la réponse
predicted_answer = model.predict(question_sequence)
print(tokenizer.sequences_to_texts(predicted_answer))

KeyboardInterrupt: Interrupted by user